In [1]:
import numpy as np
import pandas as pd

In [2]:
# load datasets
datasets = {}
years = range(2016, 2023)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')
    except:
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:11: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2019 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:13: DtypeWarning: Columns (100,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2020 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:13: DtypeWarning: Columns (96,97,98,100,102,103,104,105,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2021 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:11: DtypeWarning: Columns (117,118,119,121,123,124,125,126,137,138,139,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2022 Loading...
success


In [3]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]

In [4]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if col.startswith('eo')])

['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']


In [5]:
organizations = pd.DataFrame(columns=['eoparent'])

In [6]:
for year, dataset in datasets.items():
    organizations = pd.concat([organizations, pd.DataFrame(dataset.loc[:, 'eoparent'])], ignore_index=True)


In [7]:
organizations.drop_duplicates(inplace=True)
organizations.reset_index(drop=True, inplace=True)
# when paricipant graduated in the previous years the corresponding information about school
# (everything with 'eo') is NaN
#we drop these rows
organizations.dropna(how = 'all', inplace=True)
organizations

,eoparent
0,Відділ освіти Мелітопольської районної державн...
1,"Управління освіти, молоді та спорту Красилівсь..."
3,Управління освіти Чернівецької міської ради
4,Відділ освіти Дружківської міської ради
5,Управління освіти і науки Тернопільської міськ...
...,...
3795,"Відділ освіти, культури, молоді та спорту Ново..."
3796,"Відділ освіти, культури, молоді та спорту Біло..."
3797,"Відділ освіти, сім’ї, молоді, спорту, культури..."
3798,Гуманітарний відділ Литовезької сільської ради...


In [8]:
organizations.eoparent.unique()

array(['Відділ освіти Мелітопольської районної державної адміністрації Запорізької області',
       'Управління освіти, молоді та спорту Красилівської районної державної адміністрації',
       'Управління освіти Чернівецької міської ради', ...,
       'Відділ освіти, сім’ї, молоді, спорту, культури і туризму Ольгопільської сільської ради Вінницької області',
       'Гуманітарний відділ Литовезької сільської ради Володимир-Волинського району Волинської області',
       'Відділ освіти, культури, туризму, молоді та спорту Присиваської сільської ради'],
      dtype=object)

In [9]:
#!pip install -U duckpy
from duckpy import Client
from tqdm import tqdm
import time
tqdm.pandas()

def search_opendatabot(school_name, max_retry=20):
    """
    Search for school in opendatabot.ua
    """
    client = Client()
    opendatabot_url = ''
    retry = 0
    while not opendatabot_url and retry < max_retry:
        try:
            results = client.search(school_name, limit=20)
        except:
            time.sleep(20)
            continue
        for site in results:
            if site['url'].startswith('https://opendatabot.ua/c/'):
                opendatabot_url = site['url']
                break
        retry += 1
    return {'eoname': school_name, "url": opendatabot_url}

search_opendatabot("Комаровецький заклад загальної середньої освіти І-ІІ ступенів - заклад дошкільної освіти Барської міської ради")

{'eoname': 'Комаровецький заклад загальної середньої освіти І-ІІ ступенів - заклад дошкільної освіти Барської міської ради',
 'url': 'https://opendatabot.ua/c/26229527'}

In [10]:
opendatabot_links_new = []
for eoname in tqdm(organizations['eoparent']):
    opendatabot_links_new.append(search_opendatabot(eoname))

100%|██████████| 3799/3799 [3:24:08<00:00,  3.22s/it]    


In [12]:
opendatabot_df = pd.DataFrame(opendatabot_links_new)

In [13]:
opendatabot_df

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",
3795,"Відділ освіти, культури, молоді та спорту Біло...",
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",
3797,Гуманітарний відділ Литовезької сільської ради...,


In [14]:
opendatabot_df[opendatabot_df.url.apply(lambda x: isinstance(x, str) and len(x.strip())> 0)]

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3781,"Відділ освіти, культури, молоді та спорту Нижн...",https://opendatabot.ua/c/44041491
3782,"Відділ освіти, культури, спорту та туризму Рай...",https://opendatabot.ua/c/43915446
3783,Гуманітарний відділ виконавчого комітету Узинс...,https://opendatabot.ua/c/41973438
3784,Шабівська сільська рада,https://opendatabot.ua/c/04378043


In [15]:
opendatabot_df.to_csv('organizations_link.csv', index=False)